# T1518.001 - Security Software Discovery
Adversaries may attempt to get a listing of security software, configurations, defensive tools, and sensors that are installed on a system or in a cloud environment. This may include things such as firewall rules and anti-virus. Adversaries may use the information from [Security Software Discovery](https://attack.mitre.org/techniques/T1518/001) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

Example commands that can be used to obtain security software information are [netsh](https://attack.mitre.org/software/S0108), <code>reg query</code> with [Reg](https://attack.mitre.org/software/S0075), <code>dir</code> with [cmd](https://attack.mitre.org/software/S0106), and [Tasklist](https://attack.mitre.org/software/S0057), but other indicators of discovery behavior may be more specific to the type of software or security system the adversary is looking for. It is becoming more common to see macOS malware perform checks for LittleSnitch and KnockKnock software.

Adversaries may also utilize cloud APIs to discover the configurations of firewall rules within an environment.(Citation: Expel IO Evil in AWS) For example, the permitted IP ranges, ports or user accounts for the inbound/outbound rules of security groups, virtual firewalls established within AWS for EC2 and/or VPC instances, can be revealed by the <code>DescribeSecurityGroups</code> action with various request parameters. (Citation: DescribeSecurityGroups - Amazon Elastic Compute Cloud)

## Atomic Tests

### Atomic Test #1 - Security Software DiscoveryMethods to identify Security Software on an endpoint

when sucessfully executed, the test is going to display running processes, firewall configuration on network profiles
and specific security software.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh.exe advfirewall  show allprofiles 
netsh.exe advfirewall firewall dump
netsh.exe advfirewall show currentprofile
netsh.exe advfirewall firewall show rule name=all
netsh.exe firewall show state
netsh.exe firewall show config
sc query windefend
powershell.exe /c "Get-Process | Where-Object { $_.ProcessName -eq 'Sysmon' }"
powershell.exe /c "Get-Service | where-object {$_.DisplayName -like '*sysm*'}"
powershell.exe /c "Get-CimInstance Win32_Service -Filter 'Description = ''System Monitor service'''"
tasklist.exe
tasklist.exe | findstr /i virus
tasklist.exe | findstr /i cb
tasklist.exe | findstr /i defender
tasklist.exe | findstr /i cylance
tasklist.exe | findstr /i mc
tasklist.exe | findstr /i "virus cb defender cylance mc"
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 1

### Atomic Test #2 - Security Software Discovery - powershellMethods to identify Security Software on an endpoint

when sucessfully executed, powershell is going to processes related AV products if they are running.
Note that, depending on the privilege of current user, get-process | ?{$_.Description -like "*"} may not return the processes related to AV products of the check.
For instance, only with Administrator right, you can see the process description of McAffee processes. Hence, it is better to use get-process | ?{$_.ProcessName -like "*"},
if you know the name of those processes.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
get-process | ?{$_.Description -like "*virus*"}
get-process | ?{$_.Description -like "*carbonblack*"}
get-process | ?{$_.Description -like "*defender*"}
get-process | ?{$_.Description -like "*cylance*"}
get-process | ?{$_.Description -like "*mc*"}
get-process | ?{$_.ProcessName -like "*mc*"}
get-process | Where-Object { $_.ProcessName -eq "Sysmon" }
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 2

### Atomic Test #3 - Security Software Discovery - ps (macOS)Methods to identify Security Software on an endpoint
when sucessfully executed, command shell  is going to display AV/Security software it is running.
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
ps aux | egrep 'Little\ Snitch|CbOsxSensorService|falcond|nessusd|santad|CbDefense|td-agent|packetbeat|filebeat|auditbeat|osqueryd|BlockBlock|LuLu'
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 3

### Atomic Test #4 - Security Software Discovery - ps (Linux)Methods to identify Security Software on an endpoint
when sucessfully executed, command shell  is going to display AV/Security software it is running.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
ps aux | egrep 'falcond|nessusd|cbagentd|td-agent|packetbeat|filebeat|auditbeat|osqueryd'
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 4

### Atomic Test #5 - Security Software Discovery - pgrep (FreeBSD)Methods to identify Security Software on an endpoint
when sucessfully executed, command shell is going to display AV/Security software it is running.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
pgrep -l 'bareos-fd|icinga2|cbagentd|wazuh-agent|packetbeat|filebeat|osqueryd'
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 5

### Atomic Test #6 - Security Software Discovery - Sysmon ServiceDiscovery of an installed Sysinternals Sysmon service using driver altitude (even if the name is changed).

when sucessfully executed, the test is going to display sysmon driver instance if it is installed.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
fltmc.exe | findstr.exe 385201
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 6

### Atomic Test #7 - Security Software Discovery - AV Discovery via WMIDiscovery of installed antivirus products via a WMI query.

when sucessfully executed, the test is going to display installed AV software.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic.exe /Namespace:\\root\SecurityCenter2 Path AntiVirusProduct Get displayName /Format:List```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 7

### Atomic Test #8 - Security Software Discovery - AV Discovery via Get-CimInstance and Get-WmiObject cmdletsDiscovery of installed antivirus products via Get-CimInstance and Get-WmiObject cmdlets of powershell.

when sucessfully executed, information about installed AV software is displayed..
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
powershell Get-CimInstance -Namespace root/securityCenter2 -classname antivirusproduct
powershell Get-WmiObject -Namespace root\securitycenter2 -Class antivirusproduct
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 8

### Atomic Test #9 - Security Software Discovery - Windows Defender EnumerationWindows Defender Enumeration via different built-in windows native tools.
when sucessfully executed, information about windows defender is displayed.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Get-Service WinDefend #check the service state of Windows Defender
Get-MpComputerStatus #provides the current status of security solution elements, including Anti-Spyware, Antivirus, LoavProtection, Real-time protection, etc
Get-MpThreat #threats details that have been detected using MS Defender
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 9

### Atomic Test #10 - Security Software Discovery - Windows Firewall EnumerationEnumerates windows firewall to retrieves firewall rules from the target computer.

when sucessfully executed, details of windows firewall is displayed.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Get-NetFirewallProfile | Format-Table Name, Enabled
Get-NetFirewallSetting
Get-NetFirewallRule | select DisplayName, Enabled, Description
```

In [ ]:
Invoke-AtomicTest T1518.001 -TestNumbers 10

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as lateral movement, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

In cloud environments, additionally monitor logs for the usage of APIs that may be used to gather information about security software configurations within the environment.